In [5]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "segmentAnything"
FINAL_DATA_NAME = 'segmentAnything-full-wavelet-horizontal' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "toy-segmentAnything-resizedBlurred-normalized"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data','segmentAnything')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir
BATCH_NUM = None

In [6]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [13]:
file_list = [os.path.join(data_dir, f"{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"{RAW_DATA_SUFFIX}"))
file_names[:5]

['sa_1.npz', 'sa_1000.npz', 'sa_10002.npz', 'sa_10006.npz', 'sa_10007.npz']

In [9]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [ ]:
# '''To split large dataset into many batches, only needs to be run once'''
# k = 5000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'segmentAnything') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

1 0
1 batches created


In [14]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


toy-segmentAnything-resizedBlurred-normalized



In [15]:
import numpy as np

def npz_opener_pickle(path):
    # Load the .npz file
    with np.load(path) as data:
        arr = data['image']  # Default key if saved without naming the array

    arr = arr.astype(np.float32)
    
    # Apply jitter
    jitter = np.random.uniform(-0.5, 0.5, arr.shape)
    arr += jitter
    arr = np.clip(arr, 0, 255)
    
    return arr


# Wavelet

In [18]:
FINAL_DATA_NAME = 'segmentAnything-full-wavelet'
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "segmentAnything", f"{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "segmentAnything", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [19]:
TRANSFORM = "wavelet"
channel = "red"

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


10 layers being used


  0%|          | 0/7072 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-678.17676, -652.22003, -627.13464, ...,  635.1172 ,  642.8971 ,
        668.0058 ], dtype=float32), np.int64(3): array([-383.4756 , -350.61157, -344.27124, ...,  358.90594,  383.8398 ,
        439.1955 ], dtype=float32), np.int64(4): array([-197.38457, -194.1794 , -193.46155, ...,  213.51157,  217.4771 ,
        233.04819], dtype=float32), np.int64(5): array([-120.972855, -113.009445, -111.623   , ...,  112.34401 ,
        117.64409 ,  123.9428  ], dtype=float32), np.int64(6): array([-64.19457 , -57.760605, -56.30384 , ...,  56.519196,  57.852917,
        62.611885], dtype=float32), np.int64(7): array([-31.164955, -28.736496, -27.866615, ...,  28.118803,  29.118126,
        31.532665], dtype=float32), np.int64(8): array([-16.524029, -14.629215, -14.146246, ...,  14.208488,  14.647401,
        16.376846], dtype=float32), np.int64(9): array([-8.73206  , -7.6480093, -7.4027996, ...,  7.3780775,  7.6129622,
        8.550173 ], dtype=float32), np.int64(10): 

  0%|          | 0/7072 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-507.0229 , -474.34564, -471.00323, ...,  524.03986,  526.02856,
        558.3641 ], dtype=float32), np.int64(3): array([-365.56583, -336.74982, -331.74265, ...,  352.39352,  353.48416,
        357.52692], dtype=float32), np.int64(4): array([-235.2328 , -233.21875, -220.95795, ...,  209.02469,  214.79887,
        233.7411 ], dtype=float32), np.int64(5): array([-124.15203 , -113.04047 , -111.39036 , ...,  120.61478 ,
        123.81845 ,  125.086914], dtype=float32), np.int64(6): array([-62.513077, -58.668137, -56.83395 , ...,  57.358032,  59.190853,
        62.479527], dtype=float32), np.int64(7): array([-32.244205, -28.7179  , -27.849846, ...,  28.041912,  28.957552,
        32.651352], dtype=float32), np.int64(8): array([-16.44661 , -14.61588 , -14.133879, ...,  14.079398,  14.518905,
        16.3822  ], dtype=float32), np.int64(9): array([-8.638791 , -7.587668 , -7.298391 , ...,  7.2978473,  7.553642 ,
        8.570476 ], dtype=float32), np.int64(10): 

  0%|          | 0/7072 [00:00<?, ?it/s]

printing D
{np.int64(2): array([-276.86133, -262.0598 , -257.53687, ...,  255.44385,  296.43854,
        319.80322], dtype=float32), np.int64(3): array([-199.62662, -196.44006, -185.09824, ...,  189.14688,  206.41696,
        242.54683], dtype=float32), np.int64(4): array([-146.56544 , -120.879074, -119.34096 , ...,  117.40605 ,
        119.79411 ,  121.802536], dtype=float32), np.int64(5): array([-75.21974 , -66.07452 , -63.38159 , ...,  58.915104,  60.578575,
        67.70299 ], dtype=float32), np.int64(6): array([-39.207752, -31.50662 , -30.27081 , ...,  30.792849,  32.78789 ,
        41.850323], dtype=float32), np.int64(7): array([-21.839128, -16.54106 , -15.550946, ...,  15.565826,  16.495071,
        22.10962 ], dtype=float32), np.int64(8): array([-12.339653 ,  -8.450267 ,  -7.9618206, ...,   8.075727 ,
         8.615284 ,  14.694235 ], dtype=float32), np.int64(9): array([-7.8178825, -4.6844397, -4.3581557, ...,  4.368044 ,  4.673856 ,
        7.380308 ], dtype=float32), np.int64

In [20]:
TRANSFORM = "wavelet"
channel = "green"

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


10 layers being used


  0%|          | 0/7072 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-745.0589 , -660.1778 , -615.7639 , ...,  661.88934,  663.0331 ,
        681.46606], dtype=float32), np.int64(3): array([-380.91626, -368.74414, -360.72458, ...,  392.23517,  392.31415,
        429.35916], dtype=float32), np.int64(4): array([-216.66418, -211.31573, -210.39293, ...,  228.18356,  231.08788,
        250.41232], dtype=float32), np.int64(5): array([-127.02686 , -121.52211 , -118.35333 , ...,  121.321625,
        126.39818 ,  130.88031 ], dtype=float32), np.int64(6): array([-69.08226 , -63.17247 , -60.961338, ...,  60.534454,  62.345936,
        65.23005 ], dtype=float32), np.int64(7): array([-34.667534, -31.362051, -30.449223, ...,  30.894566,  31.819233,
        34.76724 ], dtype=float32), np.int64(8): array([-17.774971, -16.00138 , -15.527179, ...,  15.507706,  15.993097,
        17.68345 ], dtype=float32), np.int64(9): array([-9.29077 , -8.299215, -8.06871 , ...,  8.032114,  8.27582 ,
        9.106714], dtype=float32), np.int64(10): array(

  0%|          | 0/7072 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-524.0553 , -494.09946, -488.73392, ...,  503.87653,  559.31287,
        589.9274 ], dtype=float32), np.int64(3): array([-365.16125, -350.11737, -345.13196, ...,  357.7389 ,  363.19775,
        374.1172 ], dtype=float32), np.int64(4): array([-245.49539, -237.72186, -234.70297, ...,  205.97685,  207.81915,
        221.2168 ], dtype=float32), np.int64(5): array([-134.5758  , -123.814674, -120.665344, ...,  123.84921 ,
        128.50462 ,  131.19844 ], dtype=float32), np.int64(6): array([-67.52861 , -62.340614, -60.518204, ...,  61.728012,  63.871063,
        67.83798 ], dtype=float32), np.int64(7): array([-34.905376, -31.407738, -30.510624, ...,  30.778635,  31.734108,
        35.007156], dtype=float32), np.int64(8): array([-17.802124, -15.949037, -15.433606, ...,  15.419381,  15.855267,
        17.636768], dtype=float32), np.int64(9): array([-9.264736 , -8.275581 , -8.008594 , ...,  7.972843 ,  8.2182045,
        9.230653 ], dtype=float32), np.int64(10): 

  0%|          | 0/7072 [00:00<?, ?it/s]

printing D
{np.int64(2): array([-274.73557, -269.17352, -262.5015 , ...,  277.70035,  291.55402,
        303.25763], dtype=float32), np.int64(3): array([-233.23123, -206.1759 , -178.03772, ...,  198.42703,  213.8274 ,
        228.48665], dtype=float32), np.int64(4): array([-146.90253 , -130.30504 , -124.06758 , ...,  117.792   ,
        125.65044 ,  125.973206], dtype=float32), np.int64(5): array([-81.24581 , -67.99548 , -66.423325, ...,  62.18097 ,  65.64177 ,
        72.779144], dtype=float32), np.int64(6): array([-40.333416, -34.17533 , -32.44834 , ...,  33.214935,  35.696686,
        47.302155], dtype=float32), np.int64(7): array([-24.402777, -17.89554 , -16.820183, ...,  17.065174,  17.99655 ,
        23.030743], dtype=float32), np.int64(8): array([-12.817153,  -9.351709,  -8.77149 , ...,   8.869112,   9.382162,
        16.392273], dtype=float32), np.int64(9): array([-7.925021 , -5.158463 , -4.8427896, ...,  4.8412523,  5.143056 ,
        7.6745787], dtype=float32), np.int64(10): 

In [21]:
TRANSFORM = "wavelet"
channel = "blue"

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


10 layers being used


  0%|          | 0/7072 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-765.0437 , -649.1371 , -637.76855, ...,  572.7265 ,  589.72736,
        599.34625], dtype=float32), np.int64(3): array([-368.55417, -334.8588 , -334.07532, ...,  352.27148,  357.2025 ,
        386.99487], dtype=float32), np.int64(4): array([-197.84314, -197.50827, -197.4984 , ...,  200.12436,  200.4216 ,
        208.73447], dtype=float32), np.int64(5): array([-108.06545 , -102.89774 , -101.7272  , ...,  104.93591 ,
        107.5596  ,  110.073006], dtype=float32), np.int64(6): array([-55.1263  , -53.980312, -52.404346, ...,  51.65838 ,  52.828945,
        56.50515 ], dtype=float32), np.int64(7): array([-29.23072 , -26.735775, -26.069893, ...,  26.328579,  27.008957,
        29.253971], dtype=float32), np.int64(8): array([-14.868102, -13.673993, -13.259178, ...,  13.268541,  13.612808,
        14.950344], dtype=float32), np.int64(9): array([-8.133192 , -7.1145887, -6.940129 , ...,  6.9330883,  7.1278305,
        7.860303 ], dtype=float32), np.int64(10): 

  0%|          | 0/7072 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-521.03564, -417.38257, -408.82617, ...,  462.90598,  464.4419 ,
        511.82846], dtype=float32), np.int64(3): array([-322.5283 , -298.25977, -291.41797, ...,  323.11188,  345.85312,
        377.97726], dtype=float32), np.int64(4): array([-204.18643, -198.65082, -195.73509, ...,  178.83403,  180.81245,
        182.22253], dtype=float32), np.int64(5): array([-112.54578 , -105.10488 , -102.76629 , ...,   98.847786,
        102.25737 ,  105.92656 ], dtype=float32), np.int64(6): array([-56.594635, -53.44225 , -52.05373 , ...,  52.513367,  54.149654,
        57.536682], dtype=float32), np.int64(7): array([-28.767658, -26.589567, -26.012533, ...,  26.490025,  27.169966,
        29.670425], dtype=float32), np.int64(8): array([-15.124186, -13.596886, -13.176613, ...,  13.214161,  13.546956,
        14.966848], dtype=float32), np.int64(9): array([-7.8034344, -7.094029 , -6.902368 , ...,  6.876287 ,  7.0712166,
        7.9179697], dtype=float32), np.int64(10): 

  0%|          | 0/7072 [00:00<?, ?it/s]

printing D
{np.int64(2): array([-304.45337, -279.3346 , -265.54678, ...,  274.3365 ,  299.36945,
        327.89365], dtype=float32), np.int64(3): array([-199.55441, -165.4736 , -160.96179, ...,  188.58817,  190.09554,
        243.46375], dtype=float32), np.int64(4): array([-112.89511, -105.28693, -104.49629, ...,  106.43698,  108.49592,
        111.76595], dtype=float32), np.int64(5): array([-68.958145, -62.788788, -58.076134, ...,  53.760624,  58.557533,
        65.22092 ], dtype=float32), np.int64(6): array([-37.88192 , -30.515762, -28.716373, ...,  29.196152,  30.83167 ,
        36.998905], dtype=float32), np.int64(7): array([-20.029139 , -15.528746 , -14.706953 , ...,  14.973723 ,
        15.9225855,  19.909603 ], dtype=float32), np.int64(8): array([-12.904782 ,  -8.041643 ,  -7.5617924, ...,   7.6254787,
         8.077616 ,  13.29251  ], dtype=float32), np.int64(9): array([-6.4893007, -4.4856486, -4.218037 , ...,  4.204706 ,  4.4519095,
        7.1635456], dtype=float32), np.int64

In [22]:
TRANSFORM = "wavelet"
channel = "gray"

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


10 layers being used


  0%|          | 0/7072 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-704.569  , -607.08405, -559.78577, ...,  617.7823 ,  637.99677,
        665.35504], dtype=float32), np.int64(3): array([-364.742  , -343.5474 , -338.537  , ...,  366.35535,  376.45566,
        426.90564], dtype=float32), np.int64(4): array([-203.23235, -196.8623 , -195.13094, ...,  215.30226,  222.2339 ,
        240.54794], dtype=float32), np.int64(5): array([-121.53675 , -114.557434, -112.37513 , ...,  113.060326,
        119.310745,  124.02354 ], dtype=float32), np.int64(6): array([-65.573654, -59.012535, -57.014668, ...,  57.170624,  59.142414,
        62.446926], dtype=float32), np.int64(7): array([-32.364998, -29.489904, -28.604473, ...,  28.935509,  29.914597,
        33.164055], dtype=float32), np.int64(8): array([-16.908709 , -15.0862255, -14.556751 , ...,  14.6297   ,
        15.079869 ,  16.902363 ], dtype=float32), np.int64(9): array([-8.640656 , -7.8426027, -7.612022 , ...,  7.577165 ,  7.8283405,
        8.671572 ], dtype=float32), np.int64

  0%|          | 0/7072 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-505.66888, -474.3186 , -439.83362, ...,  483.0393 ,  537.469  ,
        545.4199 ], dtype=float32), np.int64(3): array([-343.49133, -327.9904 , -324.11502, ...,  340.9485 ,  347.99835,
        355.5823 ], dtype=float32), np.int64(4): array([-233.44553, -228.37988, -227.2399 , ...,  198.78986,  200.93167,
        214.31   ], dtype=float32), np.int64(5): array([-128.14407, -116.44908, -114.48159, ...,  117.39251,  119.5475 ,
        125.12438], dtype=float32), np.int64(6): array([-64.94067 , -59.36039 , -57.718063, ...,  58.41139 ,  60.474037,
        64.640816], dtype=float32), np.int64(7): array([-33.538944, -29.468643, -28.655918, ...,  28.93518 ,  29.9935  ,
        33.8389  ], dtype=float32), np.int64(8): array([-16.951433 , -15.08076  , -14.560465 , ...,  14.4853115,
        14.972651 ,  16.698717 ], dtype=float32), np.int64(9): array([-8.7051115, -7.812136 , -7.534867 , ...,  7.5076127,  7.7362676,
        8.6642275], dtype=float32), np.int64(10): 

  0%|          | 0/7072 [00:00<?, ?it/s]

printing D
{np.int64(2): array([-259.48788, -246.48027, -241.51027, ...,  255.20638,  268.2892 ,
        284.3253 ], dtype=float32), np.int64(3): array([-208.68544, -197.849  , -167.87766, ...,  189.37817,  198.16553,
        201.98859], dtype=float32), np.int64(4): array([-141.60658 , -122.759636, -120.282585, ...,  108.049614,
        116.733925,  120.30652 ], dtype=float32), np.int64(5): array([-77.76811 , -62.47676 , -61.42484 , ...,  56.51991 ,  58.395004,
        64.78164 ], dtype=float32), np.int64(6): array([-39.735985, -31.97435 , -30.519793, ...,  30.626606,  33.15329 ,
        44.658333], dtype=float32), np.int64(7): array([-23.221815, -16.775387, -15.665968, ...,  15.824578,  16.942217,
        22.574657], dtype=float32), np.int64(8): array([-12.337745 ,  -8.716613 ,  -8.197234 , ...,   8.337269 ,
         8.8252325,  15.835283 ], dtype=float32), np.int64(9): array([-7.742275 , -4.812896 , -4.5002117, ...,  4.4925904,  4.7935243,
        7.3426533], dtype=float32), np.int64